# Tutorial: Dispatcher

This notebook demonstrates how to use dispatcher.

In [1]:
import yaml
from flumodelingsuite.config_loader import load_basemodel_config_from_file, load_sampling_config_from_file, load_calibration_config_from_file
from flumodelingsuite.validation.general_validator import validate_modelset_consistency

In [2]:
from flumodelingsuite.workflow_dispatcher import (
    dispatch_builder,
    dispatch_runner,
    # dispatch_output_generator,
)

In [3]:
# import logging
# import sys

# root = logging.getLogger()
# if root.handlers:
#     root.handlers.clear()

# logging.basicConfig(
#     level=logging.INFO,
#     stream=sys.stdout,
#     format="%(asctime)s %(name)s %(levelname)s: %(message)s",
# )

***
## Loading configs

In [4]:
basemodel_config = load_basemodel_config_from_file("data/flu_round05_basemodel.yml")

In [5]:
basemodel_config_standalone = load_basemodel_config_from_file("data/basic_basemodel_standalone.yml")

In [6]:
sampling_config = load_sampling_config_from_file("data/flu_round05_modelset_sampling.yml")

In [7]:
calibration_config = load_calibration_config_from_file("data/flu_round05_modelset_calibration.yml")

***
## Validate config consistency

In [8]:
validate_modelset_consistency(basemodel_config, sampling_config)

In [9]:
validate_modelset_consistency(basemodel_config, calibration_config)

***
## Dispatcher - Builder

In [10]:
builder_output_sampling = dispatch_builder(basemodel_config=basemodel_config, sampling_config=sampling_config)

In [11]:
builder_output_calibration = dispatch_builder(basemodel_config=basemodel_config, calibration_config=calibration_config)

In [12]:
builder_output_standalone = dispatch_builder(basemodel_config=basemodel_config_standalone)

***
## Dispatcher - Runner

In [22]:
runnder_output_sampling = dispatch_runner(builder_output_sampling[0])

In [23]:
runnder_output_sampling.results

SimulationResults(trajectories=[Trajectory(compartments={'S_0-4': array([208172.,  92209.,  68860.,  66095.,  65798.,  65759.,  65752.,
        65751.,  65751.,  65751.,  65751.,  65751.,  65751.,  65751.,
        65751.,  65751.,  65751.,  65751.,  65751.,  65751.,  65751.,
        65751.,  65751.,  65751.,  65751.,  65751.,  65751.,  65751.,
        65751.,  65751.,  65751.,  65751.,  65751.]), 'S_5-17': array([375220.,  66788.,  39993.,  37578.,  37363.,  37334.,  37330.,
        37330.,  37330.,  37330.,  37330.,  37330.,  37330.,  37330.,
        37330.,  37330.,  37330.,  37330.,  37330.,  37330.,  37330.,
        37330.,  37330.,  37330.,  37330.,  37330.,  37330.,  37330.,
        37330.,  37330.,  37330.,  37330.,  37330.]), 'S_18-49': array([1486471.,  458649.,  292835.,  274418.,  272440.,  272207.,
        272186.,  272178.,  272178.,  272178.,  272178.,  272178.,
        272178.,  272178.,  272178.,  272178.,  272178.,  272178.,
        272178.,  272178.,  272178.,  272178

In [24]:
runner_output_calibration = dispatch_runner(builder_output_calibration[0])

Starting ABC-SMC with 10 particles and 2 generations

Generation 1/2 (epsilon: inf)
	Accepted 10/10 (acceptance rate: 100.00%)
	Elapsed time: 00:00:01

Generation 2/2 (epsilon: 2718.383608)
	Accepted 10/17 (acceptance rate: 58.82%)
	Elapsed time: 00:00:02


In [25]:
runner_output_calibration.results

CalibrationResults(calibration_strategy='smc', posterior_distributions={0:          Rt  seasonality_min         L  Initial_immune  start_date
0  1.374540         0.840143  0.000759        0.217557        74.0
1  1.201581         0.709483  0.000199        0.142215        42.0
2  1.328120         0.772460  0.000856        0.035014         9.0
3  1.433975         0.732228  0.000200        0.058251        14.0
4  1.289566         0.711633  0.000876        0.206137        53.0
5  1.947099         0.651146  0.000777        0.344177        16.0
6  1.422165         0.688133  0.000976        0.026789        44.0
7  1.963019         0.739509  0.000848        0.048664        35.0
8  1.701657         0.664656  0.000129        0.164164         2.0
9  1.421657         0.667672  0.000586        0.235070        47.0, 1:          Rt  seasonality_min         L  Initial_immune  start_date
0  1.546860         0.775236  0.000390        0.141593        60.0
1  1.096177         0.671108  0.000502        0.32

In [17]:
runner_output_standalone = dispatch_runner(builder_output_standalone)

In [27]:
runner_output_standalone.results

SimulationResults(trajectories=[Trajectory(compartments={'S_0-4': array([233475.,  58945.,  39540.,  38246.,  38136.,  38132.,  38131.,
        38130.,  38130.,  38130.,  38130.,  38130.,  38130.,  38130.,
        38130.,  38130.,  38130.,  38130.,  38130.,  38130.,  38130.,
        38130.,  38130.,  38130.,  38130.,  38130.,  38130.,  38130.,
        38130.,  38130.,  38130.,  38130.,  38130.,  38130.,  38130.,
        38130.,  38130.,  38130.,  38130.,  38130.,  38130.,  38130.,
        38130.,  38130.,  38130.,  38130.,  38130.,  38130.,  38130.,
        38130.,  38130.,  38130.,  38130.]), 'S_5-17': array([584580.,  85038.,  48542.,  46318.,  46066.,  46044.,  46044.,
        46044.,  46044.,  46044.,  46044.,  46044.,  46044.,  46044.,
        46044.,  46044.,  46044.,  46044.,  46044.,  46044.,  46044.,
        46044.,  46044.,  46044.,  46044.,  46044.,  46044.,  46044.,
        46044.,  46044.,  46044.,  46044.,  46044.,  46044.,  46044.,
        46044.,  46044.,  46044.,  4604